In [1]:
import os

In [2]:
%pwd

'/Users/jubril/Downloads/AI_PORTFOLIO/ML_MLFlow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/jubril/Downloads/AI_PORTFOLIO/ML_MLFlow'

In [5]:
import pandas as pd

In [7]:
data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [9]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [10]:
data.shape

(1599, 12)

In [11]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [13]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [14]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [17]:
import os
from mlProject import logger

In [19]:
class DataValidation:
    def __init__(self, config):
        """
        Initializes the DataValidation object with the provided configuration.
        
        Args:
            config: A configuration object containing:
                - unzip_data_dir: Path to the CSV data file.
                - all_schema: Dictionary of expected column names and their data types.
                - target_column.name: Name of the target column.
        """
        self.config = config

    def validate_all_columns(self) -> bool:
        """
        Validates the following:
          1. All expected columns are present in the data.
          2. The data types of each column match what is expected.
          3. The target column exists in the data.

        Returns:
            bool: True if all validations pass; False otherwise.
        """
        try:
            # Load the CSV data into a DataFrame.
            data = pd.read_csv(self.config.unzip_data_dir)
            logger.info(f"Data loaded from: {self.config.unzip_data_dir}")

            # 1. Check that all expected columns are present.
            expected_columns = list(self.config.all_schema.keys())
            missing_columns = [col for col in expected_columns if col not in data.columns]
            if missing_columns:
                logger.error(f"Missing columns: {missing_columns}")
                return False

            # 2. Validate that the data types of each column match the expected types.
            dtype_errors = []
            for col, expected_type in self.config.all_schema.items():
                # Compare the string representation of the pandas dtype with the expected type.
                if str(data[col].dtype) != expected_type:
                    dtype_errors.append(f"{col}: expected {expected_type}, got {data[col].dtype}")
            if dtype_errors:
                logger.error(f"Data type mismatches found: {dtype_errors}")
                return False

            # # 3. Check that the target column exists.
            # target_col = self.config.target_column.name
            # if target_col not in data.columns:
            #     logger.error(f"Target column '{target_col}' not found in data")
            #     return False

            # All validations passed.
            logger.info("All validations passed successfully.")
            return True

        except Exception as e:
            logger.exception("Exception occurred during validation")
            raise e


In [19]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [20]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-02-27 23:26:04,548: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-27 23:26:04,763: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-27 23:26:04,768: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-27 23:26:04,777: INFO: common: created directory at: artifacts]
[2025-02-27 23:26:04,823: INFO: common: created directory at: artifacts/data_validation]
[2025-02-27 23:26:05,243: INFO: 629506273: Data loaded from: artifacts/data_ingestion/winequality-red.csv]
[2025-02-27 23:26:05,256: ERROR: 629506273: Exception occurred during validation]
Traceback (most recent call last):
  File "/var/folders/js/yy55dqrx163d83mgr0fzwmj00000gn/T/ipykernel_31358/629506273.py", line 47, in validate_all_columns
    target_col = self.config.target_column.name
AttributeError: 'DataValidationConfig' object has no attribute 'target_column'


AttributeError: 'DataValidationConfig' object has no attribute 'target_column'